# graph.net

> TODO fill in description

In [ ]:
#| default_exp graph.net

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from abc import abstractmethod, ABC
from types import MappingProxyType
from typing import Type, Callable, Any, Tuple, Coroutine, List, Dict, NewType
import uuid
import traceback

import fbdev
from fbdev._utils import AttrContainer, TaskManager, StateCollection, StateHandler, await_multiple_events, EventCollection, abstractproperty
from fbdev.comp.packet import BasePacket, Packet
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection, BasePort, Port, PortCollection, PortID
from fbdev.comp.base_component import BaseComponent
from fbdev.graph.packet_registry import LocationUUID
from fbdev.graph.graph_spec import GraphSpec, NodeSpec, EdgeSpec
from fbdev.graph.packet_registry import TrackedPacket, PacketRegistry
from fbdev.exceptions import ComponentError

In [ ]:
#|export
Address = NewType('Address', str)

This is mostly used for debugging purposes, so that in error messages we can use the human-readable names of the locations rather than UUIDs.

In [ ]:
#|exporti
location_uuid_entitities: Dict[LocationUUID, Edge|Node] = {}

def get_location_uuid(entity) -> LocationUUID:
    uuid_int = uuid.uuid4().hex
    location_uuid_entitities[uuid_int] = entity
    
def lookup_location_uuid(uuid_int:LocationUUID) -> Edge|Node:
    return location_uuid_entitities[uuid_int]

In [ ]:
#|hide
show_doc(fbdev.graph.net.Edge)

---

### Edge

>      Edge (edge_spec:fbdev.graph.graph_spec.EdgeSpec, parent_graph_comp_proces
>            s:fbdev.graph.graph_component.GraphComponentFactory)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:

#|export
class Edge:
    _address_delimiter = '|'

    def __init__(self, edge_spec: EdgeSpec, parent_graph_comp_process:fbdev.graph.graph_component.GraphComponentFactory) -> None:
        self._edge_spec = edge_spec
        self._loc_uuid:LocationUUID = get_location_uuid(self)
        self._parent_graph_comp_process: fbdev.graph.graph_component.GraphComponentFactory = parent_graph_comp_process
        self._task_manager = TaskManager(self)
        self._edge_in_bus_task: asyncio.Task = None
        self._edge_out_bus_task: asyncio.Task = None
        if edge_spec.maxsize == 0: raise ValueError("Edge maxsize cannot be 0.")
        self._packets = asyncio.Queue(maxsize=edge_spec.maxsize) if edge_spec.maxsize else asyncio.Queue()
        
        self._states:StateCollection = StateCollection()
        self._states._add_state(StateHandler("running", False), readonly=True)
        self._states._add_state(StateHandler("full", False), readonly=True)
        self._states._add_state(StateHandler("empty", True), readonly=True)
        
    @property
    def spec(self) -> EdgeSpec: return self._edge_spec
    @property
    def id(self) -> str: return self._edge_spec.id
    @property
    def states(self): return self._states
    @property
    def loc_uuid(self) -> LocationUUID: return self._loc_uuid
    @property
    def task_manager(self) -> TaskManager: return self._task_manager
    @property
    def _packet_registry(self): return self._parent_net._packet_registry
    
    @property
    def _parent_net(self) -> BaseNode:
        if hasattr(self._parent_graph_comp_process, '_parent_node'):
            return self._parent_graph_comp_process._parent_node
        else: return None
    
    @property
    def tail(self) -> BaseNode:
        if self._parent_net is None: raise RuntimeError("Cannot use Edge.tail when its GraphComponent process does not have a parent node.")
        if self._edge_spec.tail:
            node_id = BaseNode.TOP_NODE_ID if type(self._edge_spec.tail) == GraphSpec else self._edge_spec.tail.id
            return self._parent_net.get_child_node_by_id(node_id)
        else: return None
    @property
    def tail_port(self) -> NodePort:
        if self.tail_is_connected:
            tail = self._parent_graph_comp_process if type(self._edge_spec.tail) == GraphSpec else self._parent_graph_comp_process.nodes[self._edge_spec.tail.id]
            return tail.ports[self._edge_spec._tail_node_port_id]
        else: return None
        
    @property
    def tail_is_connected(self) -> bool: return self.tail is not None
    @property
    def head_is_connected(self) -> bool: return self.head is not None
    
    @property
    def head(self) -> BaseNode:
        if self._parent_net is None: raise RuntimeError("Cannot use Edge.head when its GraphComponent process does not have a parent node.")
        if self._edge_spec.head:
            node_id = BaseNode.TOP_NODE_ID if type(self._edge_spec.head) == GraphSpec else self._edge_spec.head.id
            return self._parent_net.get_child_node_by_id(node_id)
        else: return None
    @property
    def head_port(self) -> NodePort:
        if self.head_is_connected:
            head = self._parent_graph_comp_process if type(self._edge_spec.head) == GraphSpec else self._parent_graph_comp_process.nodes[self._edge_spec.head.id]
            return head.ports[self._edge_spec._head_node_port_id]
        else: return None
        
    def start(self):
        self._edge_in_bus_task = self.task_manager.create_task(self._edge_in_bus())
        self._edge_out_bus_task = self.task_manager.create_task(self._edge_out_bus())
    
    async def stop(self):
        await self.task_manager.cancel_wait(self._edge_in_bus_task)
        await self.task_manager.cancel_wait(self._edge_out_bus_task)

    async def _edge_in_bus(self):
        packet = None
        if not self.tail_is_connected: return # TODO: Allow for attaching the tail and head after creation
        while True:
            packet_putted = self.tail_port.states.put_awaiting.get_state_event(True)
            edge_non_full = self.states.full.get_state_event(False)
            await await_multiple_events(packet_putted, edge_non_full)
            packet = await self.tail_port._get()
            if not self._packet_registry.is_registered(packet):
                packet = TrackedPacket(packet, self.tail.loc_uuid, self._packet_registry)
            self._packet_registry.register_move(packet, origin=self.tail.loc_uuid, dest=self.loc_uuid, via=self.tail_port.id)
            await self._packets.put(packet)
            packet = None
            self.states._empty.set(False)
            if self._packets.full(): self.states._empty.set(True)

    async def _edge_out_bus(self):
        packet = None
        if not self.head_is_connected: return
        while True:
            packet_getted = self.head_port.states.get_awaiting.get_state_event(True)
            edge_non_empty = self.states.empty.get_state_event(False)
            await await_multiple_events(packet_getted, edge_non_empty)
            packet = await self._packets.get()
            if packet is not None:
                await self.head_port._put(packet)
                self._packet_registry.register_move(packet, origin=self.loc_uuid, dest=self.head.loc_uuid, via=self.head_port.id) # I think this safely registers the move, as `await port._packet_queue.put(packet)` is the last await in `_put`
                packet = None
                self.states._full.set(False)
                if self._packets.empty(): self.states._empty.set(True)

    @property
    def address(self) -> Address:
        return f"{self._parent_net.address}{Edge._address_delimiter}{self.id}"
    
    def __repr__(self) -> str:
        return self.spec.__repr__()

In [ ]:
#|hide
show_doc(fbdev.graph.net.BaseNodePort)

---

### BaseNodePort

>      BaseNodePort ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class BaseNodePort(BasePort):
    _address_delimiter = ':'
    
    def __init__(self):
        super().__init__()
        
        # This is reversed from the perspective of a component, as NodePorts are something that would be interacted with *outside* of the component execution logic
        if not self.is_input_port:
            self.get = self._get_to_external
            self.get_and_consume = self._get_and_consume_to_external
        else:
            self.put = self._put_from_external
            self.put_value = self._put_value_from_external
    
    @property
    def address(self) -> Address:
        return f"{self._parent_node.address}{BaseNodePort._address_delimiter}{self.port_type}.{self.name}"

    @abstractproperty
    def spec(self) -> PortSpec: ...
    @abstractproperty
    def name(self) -> str: ...
    @abstractproperty
    def id(self) -> str: ...
    @abstractproperty
    def port_type(self) -> PortType: ...
    @abstractproperty
    def dtype(self) -> type: return ...
    @abstractproperty
    def is_input_port(self) -> bool: ...
    @abstractproperty
    def is_output_port(self) -> bool: ...
    @abstractproperty
    def data_validator(self) -> Callable[[Any], bool]: ...
    @abstractproperty
    def states(self) -> StateCollection: ...
    @abstractproperty
    def events(self) -> EventCollection: ...
    @abstractproperty
    def parent_node(self) -> BaseNode: ...
    @abstractproperty
    def packet_registry(self) -> PacketRegistry: ...
    
    @abstractmethod
    async def _put(self, packet:BasePacket): ...
    @abstractmethod
    async def _get(self) -> TrackedPacket: ...
    @abstractmethod
    async def _put_from_external(self, packet:BasePacket): ...
    @abstractmethod
    async def _get_to_external(self) -> TrackedPacket: ...
    @abstractmethod
    async def _put_value_from_external(self, val:Any): ...
    @abstractmethod
    async def _get_and_consume_to_external(self) -> Any: ...

In [ ]:
#|hide
show_doc(fbdev.graph.net.NodePort)

---

### NodePort

>      NodePort (_port:fbdev.comp.port.Port,
>                _parent_node:fbdev.graph.net.BaseNode)

*Ports in a Node will be converted to NodePorts. This is to facilitate addressing. It's mostly cosmetics.*

In [ ]:
#|exporti
class NodePort(BaseNodePort):
    """Ports in a Node will be converted to NodePorts. This is to facilitate addressing. It's mostly cosmetics."""
    _address_delimiter = ':'
    
    def __init__(self, *, _port:Port, _parent_node: BaseNode):
        self._port = _port
        self._parent_node: BaseNode = _parent_node
        super().__init__()
    
    @property
    def address(self) -> Address:
        return f"{self.parent_node.address}{NodePort._address_delimiter}{self.port_type}.{self.name}"

    @property
    def spec(self) -> PortSpec: return self._port.spec
    @property
    def name(self) -> str: return self._port.name
    @property
    def id(self) -> str: return self._port.id
    @property
    def port_type(self) -> PortType: return self._port.port_type
    @property
    def dtype(self) -> type: return self._port.dtype
    @property
    def is_input_port(self) -> bool: return self._port.is_input_port
    @property
    def is_output_port(self) -> bool: return not self._port.is_output_port
    @property
    def data_validator(self) -> Callable[[Any], bool]: return self._port.data_validator
    @property
    def states(self) -> StateCollection: return self._port.states
    @property
    def events(self) -> EventCollection: return self._port.events
    @property
    def parent_node(self) -> BaseNode: return self._parent_node
    @property
    def packet_registry(self) -> PacketRegistry: return self.parent_node._packet_registry
        
    async def _put(self, packet:BasePacket): await self._port._put(packet)
    async def _get(self) -> TrackedPacket: return await self._port._get()
    
    async def _put_from_external(self, packet:BasePacket):
        # Register that the packet is incoming from outside the net
        if not self.packet_registry.is_registered(packet):
            packet = TrackedPacket(packet, location=TrackedPacket.EXTERNAL_LOCATION, packet_registry=self.packet_registry)
        self.packet_registry.register_move(packet, origin=TrackedPacket.EXTERNAL_LOCATION, dest=self.parent_node.loc_uuid, via=self.id)
        await self._put(packet)
    
    async def _get_to_external(self) -> TrackedPacket:
        packet = await self._get()
        if not self.packet_registry.is_registered(packet):
            packet = TrackedPacket(packet, location=self.parent_node.loc_uuid, packet_registry=self.packet_registry)
        # Register that the packet is leaving the net
        self.packet_registry.register_move(packet, origin=self.parent_node.loc_uuid, dest=TrackedPacket.EXTERNAL_LOCATION, via=self.id)
        return packet
    
    async def _put_value_from_external(self, val:Any):
        await self._put_from_external(Packet(val))
        
    async def _get_and_consume_to_external(self) -> Any:
        packet: TrackedPacket = await self._get_to_external()
        return await packet.consume()

In [ ]:
#|hide
show_doc(fbdev.graph.net.NodePortCollection)

---

### NodePortCollection

>      NodePortCollection (_port_collection:fbdev.comp.port.PortCollection,
>                          _parent_node:fbdev.graph.net.BaseNode)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class NodePortCollection(PortCollection):
    def __init__(self, *, _port_collection:PortCollection, _parent_node:BaseNode):
        self._port_spec_collection: PortSpecCollection = _port_collection._port_spec_collection
        self._ports: Dict[str, NodePort] = {}
        for port_type in PortType:
            setattr(self, port_type.label, AttrContainer({}, obj_name=f"{PortCollection.__name__}.{port_type.label}"))
        for port in _port_collection.iter_ports():
            self._add_port(NodePort(_port=port, _parent_node=_parent_node))

In [ ]:
#|hide
show_doc(fbdev.graph.net.BaseNode)

---

### BaseNode

>      BaseNode (spec:fbdev.graph.graph_spec.NodeSpec,
>                parent_net:fbdev.graph.net.BaseNode|None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class BaseNode(ABC):
    TOP_NODE_ID = 'TOP'
    _address_delimiter = '->'
    
    def __init__(self, spec: NodeSpec, parent_net:BaseNode|None) -> None:
        self._spec:NodeSpec = spec
        if parent_net and not parent_net.is_net:
            raise ValueError("Node parent must be a net.")
        self._parent_net:BaseNode = parent_net
        
        self._states:StateCollection = StateCollection()
        self._states._add_state(StateHandler("started", False), readonly=True)
        self._states._add_state(StateHandler("stopped", False), readonly=True)
        
        self._loc_uuid:LocationUUID = get_location_uuid(self)
        self._task_manager = TaskManager(self)

    @property
    def spec(self) -> EdgeSpec: return self._spec
    @property
    def id(self) -> str:
        if self.parent_net: return self.spec.id
        return BaseNode.TOP_NODE_ID
    @property
    def is_net(self) -> bool:
        return issubclass(self.spec.component_type, fbdev.graph.graph_component.GraphComponentFactory)
    @property
    def is_top_node(self) -> bool: return self.parent_net is None
    @property
    def states(self): return self._states
    @property
    def parent_net(self): return self._parent_net
    @abstractproperty
    def ports(self) -> PortCollection: ...
    @property
    def port_specs(self) -> PortSpecCollection: return self.component_type.port_specs
    @property
    def loc_uuid(self) -> LocationUUID: return self._loc_uuid
    @abstractproperty
    def edge_connections(self) -> Dict[PortID, Edge]: ...
    @abstractproperty
    def internal_edge_connections(self) -> Dict[PortID, Edge]: ...
    @property
    def component_type(self) -> Type[BaseComponent]: return self.spec.component_type
    @property
    def component_name(self) -> str: return self.spec.component_name
    @abstractproperty
    def component_process(self) -> BaseComponent: ...
    @abstractproperty
    def packet_registry(self) -> PacketRegistry: ...
    @property
    def task_manager(self) -> TaskManager: return self._task_manager
    
    @property
    def nodes(self) -> MappingProxyType[str, BaseNode]:
        if not self.is_net: raise RuntimeError("Node is not a net.")
        return self.component_process.nodes
    @property
    def edges(self) -> MappingProxyType[str, Edge]:
        if not self.is_net: raise RuntimeError("Node is not a net.")
        return self.component_process.edges
    
    @abstractmethod
    async def start(self): ...
        
    @abstractmethod
    async def stop(self): ...
    
    @property
    def address(self) -> Address:
        if self.parent_net:
            return f"{self.parent_net.address}{BaseNode._address_delimiter}{self.id}"
        else: return self.TOP_NODE_ID
        
    def get_child_by_address(self, address:Address) -> BaseNode|Edge|NodePort:
        from fbdev.graph._utils.node_lookup_by_address import _get_node_child_by_address
        return _get_node_child_by_address(self, address)
    
    def get_child_node_by_id(self, node_id:str) -> BaseNode:
        
        if node_id == BaseNode.TOP_NODE_ID: return self
        else: return self.nodes[node_id]
    
    async def await_message(self, name:str):
        packet: BasePacket = await self.ports.message[name].get()
        await packet.consume()
        
    async def send_signal(self, name:str):
        await self.ports.signal[name].put(Packet.get_empty())

In [ ]:
#|hide
show_doc(fbdev.graph.net.Node)

---

### Node

>      Node (node_spec:fbdev.graph.graph_spec.NodeSpec,
>            parent_net:fbdev.graph.net.Node|None=None)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class Node(BaseNode):
    def __init__(self, node_spec: NodeSpec, parent_net:Node|None=None) -> None:
        super().__init__(node_spec, parent_net)
            
        self._component_process = self.spec.component_type()
        self._component_process._parent_node = self
        self._component_process.task_manager.subscribe(self._handle_component_process_exception)
        self._ports = NodePortCollection(_port_collection=self._component_process.ports, _parent_node=self)
        
        self._packet_registry: PacketRegistry = None
        if self.parent_net:
            self._packet_registry: PacketRegistry = self.parent_net._packet_registry
        else:
            self._packet_registry = PacketRegistry()
    
        self.__start_lock = asyncio.Lock()
        self.__stop_lock = asyncio.Lock()
        
    @property
    def ports(self) -> PortCollection: return self._ports
    @property
    def edge_connections(self) -> Dict[PortID, Edge]:
        edges = {port_id : self.parent_net.edges[edge_spec.id] for port_id, edge_spec in self.spec.edge_connections.items()}
        return edges
    @property
    def internal_edge_connections(self) -> Dict[PortID, Edge]:
        if not self.is_net: raise RuntimeError("Node is not a net.")
        edges = {port_id : self.edges[edge_id] for port_id, edge_id in self.component_type.graph._edge_connections.items()}
        return edges
    @property
    def component_process(self) -> BaseComponent: return self._component_process
    @property
    def packet_registry(self) -> PacketRegistry: return self._packet_registry
    
    async def start(self):
        async with self.__start_lock:
            if self.states.started.get(): raise RuntimeError("Node is already started.")
            if self.states.stopped.get(): raise RuntimeError("Cannot start an already stopped node.")
            await self._component_process.start()
            self.states._started.set(True)
        
    async def stop(self):
        async with self.__stop_lock:
            if not self.states.started.get(): raise RuntimeError("Node has not been started yet.")
            if self.states.stopped.get(): raise RuntimeError("Node is already stopped.")
            await self.task_manager.destroy()
            await self.component_process.stop()
            self.states._stopped.set(True)
            
    def _handle_component_process_exception(self, task:asyncio.Task, exceptions:Tuple[Exception, ...], source_trace:Tuple, tracebacks:Tuple[str, ...]):
        try: raise ComponentError() from exceptions[0]
        except ComponentError as e:
            self.task_manager.submit_exception(task, exceptions + (e,), source_trace, tracebacks)